# Preparation

In [13]:
! pip install razdel rouge transformers sentencepiece pymorphy3 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 60.3 MB/s eta 0:00:00


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from razdel import sentenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import numpy as np
import json
import re
import tarfile
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from rouge import Rouge
import nltk
from nltk.util import ngrams
import pymorphy3

In [3]:
!rm -f gazeta_raw.txt
!wget https://www.dropbox.com/s/4fxj5wmt7tjr5f2/gazeta_raw.txt

--2025-10-27 12:25:40--  https://www.dropbox.com/s/4fxj5wmt7tjr5f2/gazeta_raw.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/y8n5zf87bxpw1ob1pbkfl/gazeta_raw.txt?rlkey=zjbt0qur4f11svazg2d8ackba [following]
--2025-10-27 12:25:40--  https://www.dropbox.com/scl/fi/y8n5zf87bxpw1ob1pbkfl/gazeta_raw.txt?rlkey=zjbt0qur4f11svazg2d8ackba
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9f43cc85bf83d25783d895078f.dl.dropboxusercontent.com/cd/0/inline/C0BuMV2ANIkA6S2TkEADC7kobI5HWIUvzWqKsh4UXFMJqH0o4yQE_0-Dx_NSXs3izN1TQj3EbDLnldr3X7lfTG4RKxep1vcEZhXFiMYulql9FYc7XpkJEMGOmubuljSJZa4/file# [following]
--2025-10-27 12:25:40--  https://uc9f43cc85bf83d25783d895078f.dl.dropboxusercontent.com/cd/0/inline/C0BuMV2ANIkA6S2T

# Extractive summarization

In [4]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
def extract_summary(text, top_n=3, limit=300):
    sentences = [s.text for s in sentenize(text)]
    if len(sentences) <= top_n:
        return " ".join(sentences)[:limit]

    sentence_embeddings = model.encode(sentences)


    # sematic centroid
    centroid = np.mean(sentence_embeddings, axis=0, keepdims=True)
    centroid_similarity = cosine_similarity(sentence_embeddings, centroid).ravel()

    # positional weight
    positional_weight = np.array([1.0 - (i * 0.8 / len(sentences)) for i in range(len(sentences))])

    # combine
    combined_scores = (
        0.7 * centroid_similarity +
        0.3 * positional_weight
    )

    # get top 3 sentences with heighest score
    top_indices = np.argsort(combined_scores)[-top_n:]
    selected_sentences = [sentences[i] for i in sorted(top_indices)]

    summary = " ".join(selected_sentences)

    # cut to limit
    if len(summary) > limit:
        sentences_in_summary = [s.text for s in sentenize(summary)]
        current_length = 0
        final_sentences = []

        for sent in sentences_in_summary:
            if current_length + len(sent) + 1 <= limit:
                final_sentences.append(sent)
                current_length += len(sent) + 1
            else:
                break

        summary = " ".join(final_sentences)

    if not summary.strip():
      summary = text[:300]

    return summary.strip()


# Abstarct summarization

In [6]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
model2 = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [7]:
def abstract_summary(text, max_len=300):
    WHITESPACE_HANDLER = lambda k: re.sub(r'\s+', ' ', re.sub('\n+', ' ', k.strip()))

    if not text.strip():
        return ""

    input_text = "summarize: " + text.strip().replace("\n", " ")

    input_ids = tokenizer(
      [WHITESPACE_HANDLER(text)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=512
      )["input_ids"]

    output_ids = model2.generate(
      input_ids=input_ids,
      max_length=84,
      no_repeat_ngram_size=2,
      num_beams=4
      )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
        )

    if not summary.strip():
      summary = text[:300]

    return summary[:max_len]


# Validation

In [15]:
morph = pymorphy3.MorphAnalyzer()

In [16]:
def lemmatize_text(text):
    tokens = re.findall(r"\w+", text.lower())
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    return " ".join(lemmas)

In [17]:
def calculate_rouge_2_lemmas(references, hypotheses):
    references_lemmas = [lemmatize_text(ref) for ref in references]
    hypotheses_lemmas = [lemmatize_text(hyp) for hyp in hypotheses]
    rouge = Rouge()
    scores = rouge.get_scores(hypotheses_lemmas, references_lemmas, avg=True)
    return scores['rouge-2']

In [18]:
def evaluate_summarizers(texts_path,
                         refs_path,
                         abstract_func,
                         extract_func,
                         sample_examples=3):

    with open(texts_path, "r", encoding="utf-8") as f:
        texts = json.load(f)
    with open(refs_path, "r", encoding="utf-8") as f:
        references = json.load(f)

    assert len(texts) == len(references), "Кол-во текстов и summary не совпадает"
    print(f"Загружено {len(texts)} примеров")

    abstract_hypotheses = []
    extract_hypotheses = []

    for i, text in enumerate(texts):
        print(f"Обработка {i+1}/{len(texts)}")
        abstract_hypotheses.append(abstract_func(text))
        extract_hypotheses.append(extract_func(text))

    # Вычисляем ROUGE-2 по леммам
    abs_scores = calculate_rouge_2_lemmas(references, abstract_hypotheses)
    ext_scores = calculate_rouge_2_lemmas(references, extract_hypotheses)

    print("\nСредние метрики ROUGE-2 (по леммам):")
    print(f"Абстрактная модель:")
    print(f"  Precision: {abs_scores['p']:.4f}")
    print(f"  Recall:    {abs_scores['r']:.4f}")
    print(f"  F1:        {abs_scores['f']:.4f}")

    print(f"\nЭкстрактная модель:")
    print(f"  Precision: {ext_scores['p']:.4f}")
    print(f"  Recall:    {ext_scores['r']:.4f}")
    print(f"  F1:        {ext_scores['f']:.4f}")

    print("\nПримеры для сравнения:")
    for i in range(min(sample_examples, len(texts))):
        print(f"\n--- Пример {i+1} ---")
        print(f"Исходный текст (первые 200 символов): {texts[i][:200]}...")
        print(f"Эталонное summary: {references[i]}")
        print(f"Абстрактное summary: {abstract_hypotheses[i]}")
        print(f"Экстрактное summary: {extract_hypotheses[i]}")
        print("-" * 60)

    return abs_scores, ext_scores

In [8]:
def prepare_gazeta_data(input_path,
                        output_texts_path,
                        output_refs_path,
                        max_samples=100,
                        summary_limit=300):

    records = []
    with open(input_path, "r", encoding="utf-8") as r:
        for line in r:
            records.append(json.loads(line))


    filtered = [
        rec for rec in records
        if 'text' in rec and 'summary' in rec and len(rec['summary']) < summary_limit
    ]

    filtered = filtered[:max_samples]
    texts = [rec['text'] for rec in filtered]
    references = [rec['summary'] for rec in filtered]

    with open(output_texts_path, "w", encoding="utf-8") as f_texts:
        json.dump(texts, f_texts, ensure_ascii=False, indent=2)

    with open(output_refs_path, "w", encoding="utf-8") as f_refs:
        json.dump(references, f_refs, ensure_ascii=False, indent=2)

    print(f"Всего записей в исходном файле: {len(records)}")
    print(f"Отобрано записей (саммари < {summary_limit}): {len(filtered)}")
    print(f"Сохранено {len(texts)} текстов в: {output_texts_path}")
    print(f"Сохранено {len(references)} эталонных саммари в: {output_refs_path}")

    return None


In [19]:
input_path = 'gazeta_raw.txt'
output_texts_path = '/content/texts.json'
output_refs_path = '/content/references.json'
prepare_gazeta_data(input_path, output_texts_path, output_refs_path, max_samples=10)

Всего записей в исходном файле: 75198
Отобрано записей (саммари < 300): 10
Сохранено 10 текстов в: /content/texts.json
Сохранено 10 эталонных саммари в: /content/references.json


In [20]:
abs_scores, ext_scores = evaluate_summarizers(
    texts_path="texts.json",
    refs_path="references.json",
    abstract_func=abstract_summary,
    extract_func=extract_summary,
    sample_examples=3
)


Загружено 10 примеров
Обработка 1/10
Обработка 2/10
Обработка 3/10
Обработка 4/10
Обработка 5/10
Обработка 6/10
Обработка 7/10
Обработка 8/10
Обработка 9/10
Обработка 10/10

Средние метрики ROUGE-2 (по леммам):
Абстрактная модель:
  Precision: 0.0339
  Recall:    0.0185
  F1:        0.0235

Экстрактная модель:
  Precision: 0.0176
  Recall:    0.0208
  F1:        0.0191

Примеры для сравнения:

--- Пример 1 ---
Исходный текст (первые 200 символов): «У меня больше нет друзей», «Я хочу умереть», «Это страшный сон» — подобными сообщениями буквально кишат сегодня форумы и чаты, в которых пользователи делятся впечатлениями о проблемах с «аськой». Пер...
Эталонное summary: Популярный интернет-пейджер ICQ потерпел глобальный сбой — значительная часть пользователей по всему миру вдруг потеряла данные о своих «друзьях». На диверсию против ICQ происшедшее не похоже, хотя служба технической поддержки компании ICQ пока молчит.
Абстрактное summary: Интернет-пейджер ICQ, который считается одним из кр